In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install bert-score

In [8]:
!conda install -y gdown

Solving environment: | ^C
failed with initial frozen solve. Retrying with flexible solve.

CondaError: KeyboardInterrupt



In [ ]:
!gdown https://drive.google.com/uc?id=1Gms1XhLqBaTFkZPEqv_BGqL4NM-o0fmJ

In [ ]:
!unzip tapt_data.zip

In [9]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)
!nvidia-smi

cuda
Sun Dec  6 20:17:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.06    Driver Version: 450.51.06    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

In [10]:
#!pip install -q --force-reinstall transformers
# !pip install -q --force-reinstall git+https://github.com/adapter-hub/adapter-transformers.git
!pip install adapter-transformers
# install datasets
!pip install -q datasets

# Make sure that we have a recent version of pyarrow in the session before we continue - otherwise reboot Colab to activate it
import pyarrow
if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
    import os
    os.kill(os.getpid(), 9)

#!git clone https://github.com/huggingface/transformers
#!python transformers/utils/download_glue_data.py --tasks SST

In [11]:
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
import dataclasses
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from transformers import RobertaTokenizer, RobertaModel
from transformers import AutoTokenizer, EvalPrediction, AutoModelWithHeads, AdapterType
from transformers import (
    Trainer,
    TrainingArguments,
    glue_compute_metrics,
    glue_tasks_num_labels,
    set_seed,
    AdapterType,CONFIG_MAPPING,
    MODEL_WITH_LM_HEAD_MAPPING,
    AdapterArguments,
    AdapterConfig,
    AdapterType,
    AutoConfig,
    AutoModelWithLMHead,
    AutoTokenizer,
    DataCollatorForLanguageModeling,
    HfArgumentParser,
    LineByLineTextDataset,
    PreTrainedTokenizer,
    TextDataset,
    Trainer,
    TrainingArguments
)

from typing import Any, Iterable, List, NewType, Optional, Tuple, Union
import dataclasses
import json
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

In [12]:
!curl --create-dirs -Lo ./data/citation_intent/train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/train.jsonl
!curl --create-dirs -Lo ./data/citation_intent/dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/dev.jsonl
!curl --create-dirs -Lo ./data/citation_intent/test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/citation_intent/test.jsonl

!curl --create-dirs -Lo ./data/sciie/train.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/sciie/train.jsonl
!curl --create-dirs -Lo ./data/sciie/dev.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/sciie/dev.jsonl
!curl --create-dirs -Lo ./data/sciie/test.jsonl https://allennlp.s3-us-west-2.amazonaws.com/dont_stop_pretraining/data/sciie/test.jsonl

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  454k  100  454k    0     0   707k      0 --:--:-- --:--:-- --:--:--  706k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 30138  100 30138    0     0  76298      0 --:--:-- --:--:-- --:--:-- 76298
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38637  100 38637    0     0    98k      0 --:--:-- --:--:-- --:--:--   98k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  804k  100  804k    0     0  1289k      0 --:--:-- --:--:-- --:--:-- 1287k
  % Total    % Received % Xferd  Average Speed   Tim

In [26]:
def getDF(filename):
    with open(filename) as f:
        content = f.readlines()
    data = []
    for t in content:
      data.append(json.loads(t))
    df = pd.DataFrame(data)
    return df

sampling_fraction=0.5

filename = "data/sciie/dev.jsonl"
new_df = getDF(filename).sample(frac=sampling_fraction)

with open("flat_sciie_dev.jsonl", "w") as outfile:
    outfile.write("\n".join(new_df["text"]))

filename = "data/sciie/test.jsonl"
new_df = getDF(filename).sample(frac=sampling_fraction)

with open("flat_sciie_test.jsonl", "w") as outfile:
    outfile.write("\n".join(new_df["text"]))

filename = "data/sciie/train.jsonl"
new_df = getDF(filename).sample(frac=sampling_fraction)

with open("flat_sciie_train.jsonl", "w") as outfile:
    outfile.write("\n".join(new_df["text"]))

from transformers import LineByLineTextDataset

train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="flat_sciie_train.jsonl",
    block_size=128,
)

test_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="flat_sciie_test.jsonl",
    block_size=128,
)

val_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="flat_sciie_dev.jsonl",
    block_size=128,
)

/opt/conda/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:114: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [27]:
language = 'tapt-pfeiffer'

from transformers import AutoModelForMaskedLM, PfeifferConfig, HoulsbyConfig
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.resize_token_embeddings(len(tokenizer))
adapter_config = PfeifferConfig()
model.add_adapter(language, AdapterType.text_lang, config=adapter_config)
model.train_adapter([language])

Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at roberta-base and are newly initialized: ['lm_head.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
from transformers import DataCollatorForLanguageModeling
# from datasets import load_metric
# metric = load_metric("bertscore")

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

training_args = TrainingArguments(
    logging_steps=50, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32, 
    save_steps=130,
    evaluate_during_training=True,
    output_dir="./models/TAPT_pfeiffer",
    overwrite_output_dir=True,
    do_train=True,
    do_eval=True,
    do_predict=True,
    learning_rate=0.0001,
    num_train_epochs=100,
    save_total_limit = 1,
    gradient_accumulation_steps = 2
)
trainer = Trainer(
      model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

trainer.train()
trainer.evaluate()

# get test results
_, _, metrics = trainer.predict(test_dataset)
print(f"Test results: \n {metrics}")

/opt/conda/lib/python3.7/site-packages/transformers/training_args.py:347: FutureWarning: The `evaluate_during_training` argument is deprecated in favor of `evaluation_strategy` (which has more options)
  FutureWarning,


Step,Training Loss,Validation Loss
50,9.728006,7.027549
100,7.438903,5.729332
150,6.409431,5.549152
200,6.310161,5.516529
250,6.192893,5.344763
300,6.058325,5.350365
350,5.979995,5.177611
400,5.859517,5.296164
450,5.789614,5.187393
500,5.710713,5.168554


RuntimeError: CUDA out of memory. Tried to allocate 6.14 GiB (GPU 0; 15.90 GiB total capacity; 7.04 GiB already allocated; 3.37 GiB free; 11.66 GiB reserved in total by PyTorch)

In [29]:
!mkdir sciie_pfieffer_lang
model.save_adapter("sciie_pfieffer_lang/", "tapt-pfeiffer")

In [30]:
!ls -lh sciie_pfieffer_lang

total 155M
-rw-r--r-- 1 root root  626 Dec  6 21:28 adapter_config.json
-rw-r--r-- 1 root root  212 Dec  6 21:28 head_config.json
-rw-r--r-- 1 root root 4.6M Dec  6 21:28 pytorch_adapter.bin
-rw-r--r-- 1 root root 150M Dec  6 21:28 pytorch_model_head.bin


In [21]:
!rm -r models

In [31]:
!zip sciie_pfieffer_lang.zip sciie_pfieffer_lang/*

  adding: sciie_pfieffer_lang/adapter_config.json (deflated 56%)
  adding: sciie_pfieffer_lang/head_config.json (deflated 34%)
  adding: sciie_pfieffer_lang/pytorch_adapter.bin (deflated 7%)
  adding: sciie_pfieffer_lang/pytorch_model_head.bin (deflated 41%)


In [32]:
!ls -lh

total 187M
---------- 1 root root  263 Dec  6 20:12 __notebook_source__.ipynb
drwxr-xr-x 2 root root 4.0K Dec  6 20:49 ci_pfieffer_lang
-rw-r--r-- 1 root root  94M Dec  6 20:52 ci_pfieffer_lang.zip
drwxr-x--- 4 root root 4.0K Dec  6 20:15 data
-rw-r--r-- 1 root root  12K Dec  6 20:18 flat_ci_dev.jsonl
-rw-r--r-- 1 root root  16K Dec  6 20:18 flat_ci_test.jsonl
-rw-r--r-- 1 root root 189K Dec  6 20:18 flat_ci_train.jsonl
-rw-r--r-- 1 root root  45K Dec  6 21:00 flat_sciie_dev.jsonl
-rw-r--r-- 1 root root  94K Dec  6 21:00 flat_sciie_test.jsonl
-rw-r--r-- 1 root root 305K Dec  6 21:00 flat_sciie_train.jsonl
drwxr-xr-x 3 root root 4.0K Dec  6 21:01 models
drwxr-xr-x 6 root root 4.0K Dec  6 21:01 runs
drwxr-xr-x 2 root root 4.0K Dec  6 21:28 sciie_pfieffer_lang
-rw-r--r-- 1 root root  94M Dec  6 21:29 sciie_pfieffer_lang.zip


In [ ]:
# Download Ngrok to tunnel the tensorboard port to an external port
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Run tensorboard as well as Ngrox (for tunneling as non-blocking processes)
import os
import multiprocessing


pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
                        for cmd in [
                        f"tensorboard --logdir ./runs/ --host 0.0.0.0 --port 6006 &",
                        "./ngrok http 6006 &"
                        ]]

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [ ]:
!pip install bert-score

In [ ]:
_, _, metrics = trainer.predict(test_dataset)